Citations: Andrew Lucas: Ising formulations of many NP Problems. arXiv:1302.5843. 
https://arxiv.org/abs/1302.5843

Ising machine: Graph Partitioning

The graph partitioning problem is one of splitting a undirected graph of nodes
into two subsets of equal size such that the number of edges connecting the two
subsets is minimized. It is NP-hard.

In [5]:
import numpy as np
import scienceplots
import matplotlib.pyplot as plt
import numba
import math


In [6]:
# spin vector randomly allocated nodes to one of two subsets [-1,1]
N = 10
def generate_spins(N):
    return np.random.choice(np.array([-1, 1], dtype=np.int64), size=N) # 1D array, size. generates random sample

vector = generate_spins(N)


In [7]:
# Adjacency matrix for a graph with N nodes.
# N x N graph of 0 and 1 where 0 represents no connection, 1 represents a
# connection. 
# diagonal is all zereos. 

p = 0.5
@numba.njit
def build_adjacency_matrix(N, p):
    adjacency_matrix = np.zeros((N, N), dtype=np.int64)
    for i in range(N):
        for j in range(i + 1, N):
            if np.random.rand() < p and i != j:
                # Only add an edge if i and j are not the same node
                # and the random condition is met
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    return adjacency_matrix

adjacency_matrix = build_adjacency_matrix(N, p)
print("Adjacency Matrix:")
print(adjacency_matrix)

Adjacency Matrix:
[[0 0 1 0 0 0 1 1 1 1]
 [0 0 0 0 1 1 0 0 0 1]
 [1 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 1 0 1 0 0 1]
 [0 1 0 1 0 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 1 0 1 0 0 0 0 0]
 [1 1 1 1 0 0 0 1 0 0]]


In [8]:
# H = H(a) + H(b) <- Ising Hamiltonian 
def calc_energy(adjacency_matrix, vector):
    # H(a): Heavily penalize the two subsets NOT having the same amount of nodes.
    A = 10000 # High A to heavily penalize imbalance
    Ha = A * (np.sum(vector) ** 2)
    # H(b): Penalize the number of connections between nodes not in same subset
    Hb = 0
    B = 1 # B is the penalty for each connection between different subsets
    sum = 0
    for i in range(N):
        # only do half the matrix up to the diagonal to avoid double counting
        for j in range(i + 1, N):
            susv = adjacency_matrix[i,j] * (vector[i] * vector[j]) 
            sum += (1- susv) / 2
    Hb = sum * B
    return Ha + Hb


In [9]:
def metropolis(adjacency_matrix, vector, T, steps):
    N = len(vector)
    E_i = calc_energy(adjacency_matrix, vector)
    print("Initial Energy:", E_i)
    # Metropolis algorithm to minimize energy
    for step in range(steps):
        i = np.random.randint(N)  # Pick a random node
        # Flip the spin of the selected node
        vector[i] *= -1
        E_f = calc_energy(adjacency_matrix, vector)
        # Calculate the energy difference
        delta_E = E_f - E_i
        # Accept or reject the new state based on the Metropolis criterion
        if delta_E < 0 or np.random.rand() < np.exp(-delta_E / T):
            # Accept the new state
            E_i = E_f
        else:
            # Reject the new state, revert the spin flip
            vector[i] *= -1

    return vector, E_i


steps = 1000
T = 1.0
print("initial vector:", vector)
print("initial energy:", calc_energy(adjacency_matrix, vector))
final_vector, final_energy = metropolis(adjacency_matrix, vector, T, steps)

print("Final vector:", final_vector)
print("Final energy:", final_energy)

initial vector: [-1  1  1  1  1 -1  1 -1  1 -1]
initial energy: 40021.0
Initial Energy: 40021.0
Final vector: [-1  1  1 -1  1 -1  1 -1  1 -1]
Final energy: 22.0
